# 🔬 EEGNet GridSearch - Fair Comparison

**Same Parameters as ResNet-101:**
1. ✅ **Optimizer**: Adam, AdamW, Adagrad
2. ✅ **Activation**: ReLU, LeakyReLU
3. ✅ **L1**: [0] (same as ResNet)
4. ✅ **L2**: [0, 1e-4, 1e-3]
5. ✅ **Early Stopping**: patience=10
6. ✅ **LR Scheduler**: CosineAnnealingLR
7. ✅ **Loss**: SoftFocalLoss (gamma=3.0)
8. ✅ **Data**: Hybrid loading
9. ✅ **CV**: 3-fold

**Total: 18 configs × 3 folds = 54 runs (~9 hours)**

## 📦 CELL 1: Setup & Imports

In [1]:
import os
from pathlib import Path
import random
import time
import gc
import json
import warnings
from datetime import datetime
from itertools import product
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print("="*80)
print(" EEGNet GridSearch - Fair Comparison ".center(80, "="))
print("="*80)
print(f"\nStarted: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("✅ Seed: 42")

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

# Paths
DATA_PKG = Path("data_package")
SPEC_DIR = Path("spec_hr_out")
RESULTS_DIR = Path("eegnet_gridsearch_results")
RESULTS_DIR.mkdir(exist_ok=True)

print(f"\n✅ Results: {RESULTS_DIR}")

===================== EEGNet GridSearch - Fair Comparison ======================

Started: 2026-01-14 15:25:31
✅ Seed: 42
✅ Device: cuda:0
   GPU: NVIDIA GeForce RTX 5060 Ti

✅ Results: eegnet_gridsearch_results


## 📊 CELL 2: Load Data

In [2]:
meta_use = pd.read_csv(DATA_PKG / "meta_use.csv")
lbl = np.load(DATA_PKG / "labels.npz", allow_pickle=True)
y_soft = lbl["y_soft"]
w_conf = lbl["w_conf"]
classes = [str(c) for c in lbl["classes"]]
y_hard = y_soft.argmax(axis=1)

print("✅ Data loaded")
print(f"   Samples: {len(y_hard)}")
print(f"   Classes: {classes}")

# 3-fold CV (SAME AS RESNET & KAN)
N_FOLDS = 3
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
folds = list(skf.split(meta_use, y_hard))
print(f"\n✅ Created {N_FOLDS}-fold CV")

✅ Data loaded
   Samples: 17089
   Classes: ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

✅ Created 3-fold CV


## 🤖 CELL 3: Dataset Class

In [3]:
class SpecDataset(Dataset):
    def __init__(self, df, root_dir, y_soft, w_conf, F_target=81, T_target=600):
        self.df = df.reset_index(drop=True)
        self.root = Path(root_dir)
        self.y_soft = y_soft
        self.w_conf = w_conf
        self.F_target = F_target
        self.T_target = T_target

    def __len__(self):
        return len(self.df)

    def _center_crop_pad(self, x):
        C, F, T = x.shape
        if F >= self.F_target:
            f0 = (F - self.F_target) // 2
            x = x[:, f0:f0+self.F_target, :]
        else:
            pad = self.F_target - F
            x = np.pad(x, ((0,0),(pad//2, pad-pad//2),(0,0)), mode="constant")
        if T >= self.T_target:
            t0 = (T - self.T_target) // 2
            x = x[:, :, t0:t0+self.T_target]
        else:
            pad = self.T_target - T
            x = np.pad(x, ((0,0),(0,0),(pad//2, pad-pad//2)), mode="constant")
        return x.copy()

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        eid = int(row.eeg_id)
        
        npz = np.load(self.root / f"{eid}_hr.npz")
        x = npz["x"]
        x = self._center_crop_pad(x)
        x = torch.from_numpy(x).float()
        x = F.interpolate(x.unsqueeze(0), size=(224, 224),
                          mode="bilinear", align_corners=False).squeeze(0)
        
        y = torch.from_numpy(self.y_soft[self.df.index[idx]]).float()
        w = torch.tensor(self.w_conf[self.df.index[idx]], dtype=torch.float32)
        
        return x, y, w

print("✅ Dataset ready")

✅ Dataset ready


## 🏗️ CELL 4: EEGNet Model (Configurable Activation)

In [4]:
class EEGNet_Configurable(nn.Module):
    """EEGNet with configurable activation function"""
    
    def __init__(self, n_classes=6, n_channels=4, activation='relu'):
        super().__init__()
        
        # Store activation type
        self.activation_name = activation
        
        # Block 1: Temporal convolution
        self.conv1 = nn.Conv2d(n_channels, 8, kernel_size=(1, 64), 
                              padding=(0, 32), bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        
        # Block 2: Depthwise spatial convolution
        self.dw_conv = nn.Conv2d(8, 16, kernel_size=(224, 1), 
                                groups=8, bias=False)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.AvgPool2d(kernel_size=(1, 4))
        self.dropout1 = nn.Dropout(0.25)
        
        # Block 3: Separable convolution
        self.sep_conv = nn.Conv2d(16, 16, kernel_size=(1, 16), 
                                 padding=(0, 8), bias=False)
        self.bn3 = nn.BatchNorm2d(16)
        self.pool2 = nn.AvgPool2d(kernel_size=(1, 8))
        self.dropout2 = nn.Dropout(0.25)
        
        # Calculate flattened size
        self.flat_size = 16 * 7
        
        # Classifier
        self.fc1 = nn.Linear(self.flat_size, 64)
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, n_classes)
    
    def get_activation(self):
        """Return activation function based on config"""
        if self.activation_name == 'relu':
            return F.relu
        elif self.activation_name == 'leakyrelu':
            return lambda x: F.leaky_relu(x, negative_slope=0.01)
        else:
            return F.relu  # default
    
    def forward(self, x):
        act = self.get_activation()
        
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = act(x)
        
        # Block 2
        x = self.dw_conv(x)
        x = self.bn2(x)
        x = act(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # Block 3
        x = self.sep_conv(x)
        x = self.bn3(x)
        x = act(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # Classifier
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = act(x)
        x = self.dropout3(x)
        x = self.fc2(x)
        
        return x

print("✅ EEGNet model ready")
print("   Supports: ReLU, LeakyReLU")

✅ EEGNet model ready
   Supports: ReLU, LeakyReLU


## 🎯 CELL 5: SoftFocalLoss

In [5]:
class SoftFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=3.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, logits, soft_targets, sample_weights=None):
        hard_targets = soft_targets.argmax(dim=1)
        probs = F.softmax(logits, dim=1)
        p_t = probs.gather(1, hard_targets.unsqueeze(1)).squeeze(1)
        ce_loss = -(soft_targets * F.log_softmax(logits, dim=1)).sum(dim=1)
        focal_weight = ((1 - p_t) ** self.gamma)
        loss = focal_weight * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[hard_targets]
            loss = alpha_t * loss
        
        if sample_weights is not None:
            loss = loss * sample_weights
        
        return loss.mean()

print("✅ SoftFocalLoss ready")

✅ SoftFocalLoss ready


## 📦 CELL 6: Hybrid Data Loader

In [6]:
def create_hybrid_loader(fold=0, target_ratio=0.4, weight_power=3.0, batch_size=16):
    tr_idx, va_idx = folds[fold]
    df_tr = meta_use.iloc[tr_idx]
    y_soft_tr, w_conf_tr = y_soft[tr_idx], w_conf[tr_idx]
    
    y_hard = y_soft_tr.argmax(axis=1)
    counts = np.bincount(y_hard, minlength=6)
    target = int(counts.max() * target_ratio)
    
    indices_add = []
    for i in range(6):
        mask = y_hard == i
        if mask.sum() < target:
            idx = np.where(mask)[0]
            n_add = target - mask.sum()
            indices_add.extend(np.random.choice(idx, n_add, replace=True))
    
    all_idx = np.concatenate([np.arange(len(y_hard)), indices_add])
    np.random.shuffle(all_idx)
    
    df_tr_over = df_tr.iloc[all_idx].reset_index(drop=True)
    y_soft_over, w_conf_over = y_soft_tr[all_idx], w_conf_tr[all_idx]
    
    y_hard_over = y_soft_over.argmax(axis=1)
    counts_over = np.bincount(y_hard_over, minlength=6)
    
    weights = (len(y_hard_over) / (counts_over + 1)) ** weight_power
    weights = torch.FloatTensor(weights / weights.sum() * 6)
    
    sample_weights = weights[y_hard_over].numpy()
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    ds_tr = SpecDataset(df_tr_over, SPEC_DIR, y_soft_over, w_conf_over)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, sampler=sampler, num_workers=0)
    
    ds_va = SpecDataset(meta_use.iloc[va_idx], SPEC_DIR, y_soft[va_idx], w_conf[va_idx])
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return dl_tr, dl_va, weights

print("✅ Hybrid loader ready")

✅ Hybrid loader ready


## 📈 CELL 7: Evaluation

In [7]:
@torch.no_grad()
def evaluate_full(model, loader):
    model.eval()
    preds, targets = [], []
    
    for x, y, w in loader:
        x = x.to(device)
        logits = model(x)
        preds.append(logits.argmax(1).cpu().numpy())
        targets.append(y.argmax(1).cpu().numpy())
    
    y_pred = np.concatenate(preds)
    y_true = np.concatenate(targets)
    
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
    }

print("✅ Evaluation ready")

✅ Evaluation ready


## 🏋️ CELL 8: Training Function

In [8]:
def train_one_config(fold, optimizer_name, activation, l1_lambda, l2_lambda,
                     lr=3e-4, batch_size=16, epochs=30, patience=10):
    import sys
    
    # Data
    print(f"      [1/5] Data...", end=" ", flush=True)
    t0 = time.time()
    dl_tr, dl_va, class_weights = create_hybrid_loader(fold=fold, batch_size=batch_size)
    print(f"✓ ({time.time()-t0:.1f}s)", flush=True)
    
    # Model
    print(f"      [2/5] Model ({activation})...", end=" ", flush=True)
    t0 = time.time()
    model = EEGNet_Configurable(
        n_classes=6, 
        n_channels=4,
        activation=activation
    ).to(device)
    print(f"✓ ({time.time()-t0:.1f}s)", flush=True)
    
    # Optimizer
    print(f"      [3/5] Optimizer ({optimizer_name}, L2={l2_lambda:.0e})...", end=" ", flush=True)
    if optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda)
    elif optimizer_name == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2_lambda)
    elif optimizer_name == 'adagrad':
        optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=l2_lambda)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")
    print(f"✓", flush=True)
    
    # Loss & Scheduler
    print(f"      [4/5] Loss & Scheduler...", end=" ", flush=True)
    criterion = SoftFocalLoss(alpha=class_weights.to(device), gamma=3.0)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    print(f"✓", flush=True)
    
    # Training
    print(f"      [5/5] Training (patience={patience}, L1={l1_lambda:.0e})...", flush=True)
    best_f1, best_state, no_improve = 0.0, None, 0
    
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss, n = 0.0, 0
        
        for x, y, w in dl_tr:
            x, y, w = x.to(device), y.to(device), w.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y, w)
            
            # L1 Regularization (SAME AS RESNET)
            if l1_lambda > 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss = loss + l1_lambda * l1_norm
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item() * x.size(0)
            n += x.size(0)
        
        train_loss /= n
        val_results = evaluate_full(model, dl_va)
        scheduler.step()
        
        # Early stopping
        if val_results['f1'] > best_f1:
            best_f1 = val_results['f1']
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"        Early stop at epoch {epoch}", flush=True)
                break
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"        Epoch {epoch:2d}: F1={val_results['f1']:.4f}, Loss={train_loss:.4f}", flush=True)
        
        if epoch % 5 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    if best_state:
        model.load_state_dict(best_state)
    
    final_results = evaluate_full(model, dl_va)
    
    del model, optimizer, scheduler, dl_tr, dl_va
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_results

print("✅ Training function ready")
print("   Optimizers: Adam, AdamW, Adagrad")
print("   Activations: ReLU, LeakyReLU")
print("   L1/L2 regularization supported")
print("   Early stopping: patience=10")

✅ Training function ready
   Optimizers: Adam, AdamW, Adagrad
   Activations: ReLU, LeakyReLU
   L1/L2 regularization supported
   Early stopping: patience=10


## 🔍 CELL 9: Grid Configuration (SAME AS RESNET)

In [9]:
print("\n" + "="*80)
print(" EEGNET GRIDSEARCH - SAME AS RESNET-101 ".center(80, "="))
print("="*80)

# EXACT SAME PARAMS AS RESNET-101
param_grid = {
    'optimizer': ['adam', 'adamw', 'adagrad'],  # 3 - SAME
    'activation': ['relu', 'leakyrelu'],        # 2 - SAME
    'l1_lambda': [0],                           # 1 - SAME (no L1)
    'l2_lambda': [0, 1e-4, 1e-3],              # 3 - SAME
}

fixed_params = {
    'lr': 3e-4,
    'batch_size': 16,
    'epochs': 30,
    'patience': 10,
}

keys = list(param_grid.keys())
values = list(param_grid.values())
combinations = list(product(*values))

print("\n📋 HYPERPARAMETER GRID:")
print("-"*80)
print(f"  Optimizer:   {param_grid['optimizer']}")
print(f"  Activation:  {param_grid['activation']}")
print(f"  L1 lambda:   {param_grid['l1_lambda']}")
print(f"  L2 lambda:   {param_grid['l2_lambda']}")

print("\n📊 GRIDSEARCH STATISTICS:")
print("-"*80)
print(f"  Total combinations: {len(combinations)}")
print(f"  Folds per config:   {N_FOLDS}")
print(f"  Total trainings:    {len(combinations) * N_FOLDS}")
print(f"  Est. time per run:  ~10 min (EEGNet is smaller)")
print(f"  Est. total time:    ~{len(combinations) * N_FOLDS * 10 / 60:.1f} hours")

print("\n📝 ALL COMBINATIONS:")
print("-"*80)
for i, combo in enumerate(combinations, 1):
    params = dict(zip(keys, combo))
    print(f"  {i:2d}. {params['optimizer']:7s} + {params['activation']:10s} + "
          f"L1={params['l1_lambda']:.0e} + L2={params['l2_lambda']:.0e}")

print("\n🎯 TARGETS TO BEAT:")
print("-"*80)
print("  EEGNet (baseline, no tuning): F1 = 0.3281")
print("  ResNet-101 (tuned):           F1 = 0.5585")
print("  KAN (tuned):                  F1 = ??? (running)")

print("\n⏱️  TIMELINE:")
print("-"*80)
current_time = datetime.now()
finish_time = current_time + pd.Timedelta(hours=len(combinations) * N_FOLDS * 10 / 60)
print(f"  Start:  {current_time.strftime('%Y-%m-%d %H:%M')}")
print(f"  Finish: {finish_time.strftime('%Y-%m-%d %H:%M')} (approx)")

print("\n💾 AUTO-SAVE:")
print("-"*80)
print(f"  {RESULTS_DIR}/eegnet_gridsearch_progress.json")
print(f"  {RESULTS_DIR}/eegnet_gridsearch_final.json")


==================== EEGNET GRIDSEARCH - SAME AS RESNET-101 ====================

📋 HYPERPARAMETER GRID:
--------------------------------------------------------------------------------
  Optimizer:   ['adam', 'adamw', 'adagrad']
  Activation:  ['relu', 'leakyrelu']
  L1 lambda:   [0]
  L2 lambda:   [0, 0.0001, 0.001]

📊 GRIDSEARCH STATISTICS:
--------------------------------------------------------------------------------
  Total combinations: 18
  Folds per config:   3
  Total trainings:    54
  Est. time per run:  ~10 min (EEGNet is smaller)
  Est. total time:    ~9.0 hours

📝 ALL COMBINATIONS:
--------------------------------------------------------------------------------
   1. adam    + relu       + L1=0e+00 + L2=0e+00
   2. adam    + relu       + L1=0e+00 + L2=1e-04
   3. adam    + relu       + L1=0e+00 + L2=1e-03
   4. adam    + leakyrelu  + L1=0e+00 + L2=0e+00
   5. adam    + leakyrelu  + L1=0e+00 + L2=1e-04
   6. adam    + leakyrelu  + L1=0e+00 + L2=1e-03
   7. adamw   + rel

## 🚀 CELL 10: Run GridSearch

In [10]:
all_results = []
start_time = time.time()

print("\n" + "="*80)
print(" STARTING GRIDSEARCH ".center(80, "="))
print("="*80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

for combo_idx, combo in enumerate(combinations, 1):
    params = dict(zip(keys, combo))
    
    print("\n" + "="*80)
    print(f" CONFIG {combo_idx}/{len(combinations)} ".center(80, "="))
    print("="*80)
    print(f"  Optimizer: {params['optimizer']}")
    print(f"  Activation: {params['activation']}")
    print(f"  L1: {params['l1_lambda']:.0e}")
    print(f"  L2: {params['l2_lambda']:.0e}")
    print("-"*80)
    
    fold_results = []
    
    for fold in range(N_FOLDS):
        print(f"\n    Fold {fold+1}/{N_FOLDS}...", flush=True)
        fold_start = time.time()
        
        try:
            result = train_one_config(
                fold=fold,
                optimizer_name=params['optimizer'],
                activation=params['activation'],
                l1_lambda=params['l1_lambda'],
                l2_lambda=params['l2_lambda'],
                **fixed_params
            )
            fold_results.append(result)
            print(f"\n    ✓ Fold {fold+1}: F1={result['f1']:.4f} ({(time.time()-fold_start)/60:.1f} min)", flush=True)
        except Exception as e:
            print(f"\n    ❌ Error: {e}", flush=True)
            fold_results.append({'f1': 0.0, 'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0})
    
    mean_metrics = {
        'f1': np.mean([r['f1'] for r in fold_results]),
        'accuracy': np.mean([r['accuracy'] for r in fold_results]),
        'precision': np.mean([r['precision'] for r in fold_results]),
        'recall': np.mean([r['recall'] for r in fold_results]),
        'f1_std': np.std([r['f1'] for r in fold_results]),
    }
    
    result_entry = {
        'config_id': combo_idx,
        'params': params,
        'mean_metrics': mean_metrics,
        'fold_results': fold_results,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    }
    all_results.append(result_entry)
    
    print(f"\n  Mean F1: {mean_metrics['f1']:.4f} ± {mean_metrics['f1_std']:.4f}")
    print(f"  Mean Acc: {mean_metrics['accuracy']:.4f}")
    
    # Auto-save
    with open(RESULTS_DIR / 'eegnet_gridsearch_progress.json', 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    print(f"  💾 Saved", flush=True)

# Final save
with open(RESULTS_DIR / 'eegnet_gridsearch_final.json', 'w') as f:
    json.dump({
        'all_results': all_results,
        'param_grid': param_grid,
        'fixed_params': fixed_params,
        'total_time_hours': (time.time() - start_time) / 3600,
    }, f, indent=2, default=str)

print("\n" + "="*80)
print(" GRIDSEARCH COMPLETE ".center(80, "="))
print("="*80)
print(f"Total time: {(time.time()-start_time)/3600:.2f} hours")


============================= STARTING GRIDSEARCH ==============================
Started: 2026-01-14 15:26:57


================================= CONFIG 1/18 ==================================
  Optimizer: adam
  Activation: relu
  L1: 0e+00
  L2: 0e+00
--------------------------------------------------------------------------------

    Fold 1/3...
      [1/5] Data... ✓ (0.0s)
      [2/5] Model (relu)... ✓ (0.1s)
      [3/5] Optimizer (adam, L2=0e+00)... ✓
      [4/5] Loss & Scheduler... ✓
      [5/5] Training (patience=10, L1=0e+00)...
        Epoch  1: F1=0.2419, Loss=0.9869
        Epoch  5: F1=0.3200, Loss=0.7310
        Epoch 10: F1=0.3528, Loss=0.6470
        Epoch 15: F1=0.3753, Loss=0.5995
        Epoch 20: F1=0.3728, Loss=0.5817
        Epoch 25: F1=0.3827, Loss=0.5663
        Epoch 30: F1=0.3815, Loss=0.5555

    ✓ Fold 1: F1=0.3838 (37.8 min)

    Fold 2/3...
      [1/5] Data... ✓ (0.0s)
      [2/5] Model (relu)... ✓ (0.0s)
      [3/5] Optimizer (adam, L2=0e+00)... ✓
     

## 📊 CELL 11: Analyze Results

In [1]:
sorted_results = sorted(all_results, key=lambda x: x['mean_metrics']['f1'], reverse=True)

print("\n" + "="*80)
print(" EEGNET GRIDSEARCH RESULTS ".center(80, "="))
print("="*80)

print("\n🏆 TOP 10 CONFIGURATIONS:")
print("="*80)
print(f"{'Rank':<6} {'Optimizer':>10} {'Activation':>12} {'L1':>8} {'L2':>8} {'F1':>10} {'Acc':>8}")
print("-"*80)

for i, result in enumerate(sorted_results[:10], 1):
    p = result['params']
    m = result['mean_metrics']
    print(f"{i:<6} {p['optimizer']:>10} {p['activation']:>12} {p['l1_lambda']:>8.0e} "
          f"{p['l2_lambda']:>8.0e} {m['f1']:>10.4f} {m['accuracy']:>8.4f}")

best = sorted_results[0]
print("\n" + "="*80)
print(" BEST EEGNET CONFIGURATION ".center(80, "="))
print("="*80)
print(f"  Optimizer:  {best['params']['optimizer']}")
print(f"  Activation: {best['params']['activation']}")
print(f"  L1:         {best['params']['l1_lambda']:.0e}")
print(f"  L2:         {best['params']['l2_lambda']:.0e}")
print(f"\n  F1:       {best['mean_metrics']['f1']:.4f} ± {best['mean_metrics']['f1_std']:.4f}")
print(f"  Accuracy: {best['mean_metrics']['accuracy']:.4f}")

print("\n" + "="*80)
print(" FINAL COMPARISON ".center(80, "="))
print("="*80)
print(f"  EEGNet (baseline):     F1 = 0.3281,  Acc = 0.3154")
print(f"  EEGNet (tuned):        F1 = {best['mean_metrics']['f1']:.4f}, Acc = {best['mean_metrics']['accuracy']:.4f}")
print(f"  KAN (tuned):           F1 = ???,     Acc = ???")
print(f"  ResNet-101 (tuned):    F1 = 0.5585,  Acc = 0.5921")

if best['mean_metrics']['f1'] > 0.3281:
    improvement = ((best['mean_metrics']['f1'] - 0.3281) / 0.3281) * 100
    print(f"\n  ✅ EEGNet improved by {improvement:.1f}% with GridSearch!")
else:
    print("\n  ⚠️  EEGNet did not improve")

print("\n💾 Results saved to:")
print(f"   {RESULTS_DIR}/eegnet_gridsearch_final.json")

NameError: name 'all_results' is not defined

In [6]:
import json
from pathlib import Path
import pandas as pd

# Path spesifik ke file
file_path = r"C:\Users\numpppy\Downloads\hms-harmful-brain-activity-classification\eegnet_gridsearch_results\eegnet_gridsearch_final.json"

print("🔍 Loading file...")
print(f"📁 {file_path}\n")

# Load data
with open(file_path, 'r') as f:
    data = json.load(f)

# Extract results
all_results = data.get('sorted_results', data.get('results', []))

print("="*80)
print(f"✅ Successfully loaded {len(all_results)} configurations!")
print("="*80)

# Quick preview - Top 5
print("\n🏆 TOP 5 CONFIGURATIONS:")
print("="*80)
print(f"{'Rank':<6} {'Optimizer':>10} {'Activation':>12} {'L1':>8} {'L2':>8} {'F1':>10} {'Acc':>8}")
print("-"*80)

sorted_results = sorted(
    all_results, 
    key=lambda x: x.get('mean_metrics', {}).get('f1', 0), 
    reverse=True
)

for i, result in enumerate(sorted_results[:5], 1):
    p = result.get('params', {})
    m = result.get('mean_metrics', {})
    print(f"{i:<6} {p.get('optimizer', 'N/A'):>10} "
          f"{p.get('activation', 'N/A'):>12} "
          f"{p.get('l1_lambda', 0):>8.0e} "
          f"{p.get('l2_lambda', 0):>8.0e} "
          f"{m.get('f1', 0):>10.4f} "
          f"{m.get('accuracy', 0):>8.4f}")

# Best configuration details
best = sorted_results[0]
print("\n" + "="*80)
print(" 🎯 BEST CONFIGURATION ".center(80, "="))
print("="*80)
print(f"  Optimizer:  {best['params'].get('optimizer', 'N/A')}")
print(f"  Activation: {best['params'].get('activation', 'N/A')}")
print(f"  L1 Lambda:  {best['params'].get('l1_lambda', 0):.0e}")
print(f"  L2 Lambda:  {best['params'].get('l2_lambda', 0):.0e}")

m = best.get('mean_metrics', {})
print(f"\n  📊 Performance:")
print(f"     F1:        {m.get('f1', 0):.4f} ± {m.get('f1_std', 0):.4f}")
print(f"     Accuracy:  {m.get('accuracy', 0):.4f} ± {m.get('accuracy_std', 0):.4f}")
print(f"     Precision: {m.get('precision', 0):.4f} ± {m.get('precision_std', 0):.4f}")
print(f"     Recall:    {m.get('recall', 0):.4f} ± {m.get('recall_std', 0):.4f}")

# Comparison
print("\n" + "="*80)
print(" 📈 MODEL COMPARISON ".center(80, "="))
print("="*80)
print(f"{'Model':<25} {'F1':>10} {'Accuracy':>10}")
print("-"*80)
print(f"{'EEGNet (baseline)':<25} {0.3281:>10.4f} {0.3154:>10.4f}")
print(f"{'EEGNet (tuned)':<25} {m.get('f1', 0):>10.4f} {m.get('accuracy', 0):>10.4f}")
print(f"{'ResNet-101 (tuned)':<25} {0.5585:>10.4f} {0.5921:>10.4f}")

# Improvement analysis
baseline_f1 = 0.3281
tuned_f1 = m.get('f1', 0)

if tuned_f1 > baseline_f1:
    improvement = ((tuned_f1 - baseline_f1) / baseline_f1) * 100
    print(f"\n  ✅ EEGNet improved by {improvement:.2f}% with GridSearch!")
else:
    decline = ((baseline_f1 - tuned_f1) / baseline_f1) * 100
    print(f"\n  ❌ EEGNet declined by {decline:.2f}%")

# Gap to best model
resnet_f1 = 0.5585
gap = resnet_f1 - tuned_f1
gap_pct = (gap / resnet_f1) * 100
print(f"  📉 Gap to ResNet-101: {gap:.4f} ({gap_pct:.1f}%)")

print("\n" + "="*80)

# Convert to DataFrame for deeper analysis
print("\n📊 Creating detailed analysis...")

records = []
for r in all_results:
    record = {**r.get('params', {}), **r.get('mean_metrics', {})}
    records.append(record)

df = pd.DataFrame(records)

print("\n📈 HYPERPARAMETER IMPACT ANALYSIS")
print("="*80)

# Optimizer impact
if 'optimizer' in df.columns:
    print("\n🔧 OPTIMIZER:")
    print("-"*80)
    opt_stats = df.groupby('optimizer')['f1'].agg(['mean', 'std', 'max', 'count'])
    opt_stats = opt_stats.sort_values('mean', ascending=False)
    print(opt_stats)

# Activation impact
if 'activation' in df.columns:
    print("\n⚡ ACTIVATION FUNCTION:")
    print("-"*80)
    act_stats = df.groupby('activation')['f1'].agg(['mean', 'std', 'max', 'count'])
    act_stats = act_stats.sort_values('mean', ascending=False)
    print(act_stats)

# L1/L2 impact
if 'l1_lambda' in df.columns and 'l2_lambda' in df.columns:
    print("\n🎛️ REGULARIZATION:")
    print("-"*80)
    print("L1 Lambda:")
    l1_stats = df.groupby('l1_lambda')['f1'].agg(['mean', 'std', 'max'])
    print(l1_stats.sort_values('mean', ascending=False))
    print("\nL2 Lambda:")
    l2_stats = df.groupby('l2_lambda')['f1'].agg(['mean', 'std', 'max'])
    print(l2_stats.sort_values('mean', ascending=False))

# Statistics
print("\n📊 OVERALL STATISTICS")
print("="*80)
print(f"Total configurations tested: {len(df)}")
print(f"\nF1 Score Distribution:")
print(f"  Best:   {df['f1'].max():.4f}")
print(f"  Worst:  {df['f1'].min():.4f}")
print(f"  Mean:   {df['f1'].mean():.4f}")
print(f"  Median: {df['f1'].median():.4f}")
print(f"  Std:    {df['f1'].std():.4f}")

print("\n" + "="*80)
print("✅ Analysis complete!")
print("="*80)

🔍 Loading file...
📁 C:\Users\numpppy\Downloads\hms-harmful-brain-activity-classification\eegnet_gridsearch_results\eegnet_gridsearch_final.json

✅ Successfully loaded 0 configurations!

🏆 TOP 5 CONFIGURATIONS:
Rank    Optimizer   Activation       L1       L2         F1      Acc
--------------------------------------------------------------------------------


IndexError: list index out of range

In [8]:
import json
from pathlib import Path
import pandas as pd

# Load file
file_path = r"C:\Users\numpppy\Downloads\hms-harmful-brain-activity-classification\eegnet_gridsearch_results\eegnet_gridsearch_final.json"

print("🔍 Loading file...")
print(f"📁 {file_path}\n")

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract results (KEY YANG BENAR: 'all_results')
all_results = data['all_results']

print("="*80)
print(f"✅ Successfully loaded {len(all_results)} configurations!")
print("="*80)

# Sort by F1 score
sorted_results = sorted(
    all_results, 
    key=lambda x: x.get('mean_metrics', {}).get('f1', 0), 
    reverse=True
)

# Display TOP 10
print("\n🏆 TOP 10 CONFIGURATIONS:")
print("="*80)
print(f"{'Rank':<6} {'ID':<5} {'Optimizer':>10} {'Activation':>12} {'L1':>8} {'L2':>8} {'F1':>10} {'Acc':>8}")
print("-"*80)

for i, result in enumerate(sorted_results[:10], 1):
    p = result.get('params', {})
    m = result.get('mean_metrics', {})
    config_id = result.get('config_id', 'N/A')
    
    print(f"{i:<6} {config_id:<5} {p.get('optimizer', 'N/A'):>10} "
          f"{p.get('activation', 'N/A'):>12} "
          f"{p.get('l1_lambda', 0):>8.0e} "
          f"{p.get('l2_lambda', 0):>8.0e} "
          f"{m.get('f1', 0):>10.4f} "
          f"{m.get('accuracy', 0):>8.4f}")

# Best configuration details
best = sorted_results[0]
print("\n" + "="*80)
print(" 🎯 BEST CONFIGURATION ".center(80, "="))
print("="*80)
print(f"  Config ID:  {best.get('config_id', 'N/A')}")
print(f"  Optimizer:  {best['params'].get('optimizer', 'N/A')}")
print(f"  Activation: {best['params'].get('activation', 'N/A')}")
print(f"  L1 Lambda:  {best['params'].get('l1_lambda', 0):.0e}")
print(f"  L2 Lambda:  {best['params'].get('l2_lambda', 0):.0e}")

m = best.get('mean_metrics', {})
print(f"\n  📊 Performance:")
print(f"     F1:        {m.get('f1', 0):.4f} ± {m.get('f1_std', 0):.4f}")
print(f"     Accuracy:  {m.get('accuracy', 0):.4f} ± {m.get('accuracy_std', 0):.4f}")
print(f"     Precision: {m.get('precision', 0):.4f} ± {m.get('precision_std', 0):.4f}")
print(f"     Recall:    {m.get('recall', 0):.4f} ± {m.get('recall_std', 0):.4f}")
if 'loss' in m:
    print(f"     Loss:      {m.get('loss', 0):.4f} ± {m.get('loss_std', 0):.4f}")

# Grid Search Info
print("\n" + "="*80)
print(" 🔧 GRID SEARCH CONFIGURATION ".center(80, "="))
print("="*80)
print(f"  Total configs tested: {len(all_results)}")
print(f"  Total time: {data.get('total_time_hours', 'N/A'):.2f} hours" if 'total_time_hours' in data else "")
print(f"\n  Parameter Grid:")
for param, values in data.get('param_grid', {}).items():
    print(f"    {param}: {values}")
print(f"\n  Fixed Parameters:")
for param, value in data.get('fixed_params', {}).items():
    print(f"    {param}: {value}")

# Comparison
print("\n" + "="*80)
print(" 📈 MODEL COMPARISON ".center(80, "="))
print("="*80)
print(f"{'Model':<25} {'F1':>10} {'Accuracy':>10}")
print("-"*80)
print(f"{'EEGNet (baseline)':<25} {0.3281:>10.4f} {0.3154:>10.4f}")
print(f"{'EEGNet (tuned)':<25} {m.get('f1', 0):>10.4f} {m.get('accuracy', 0):>10.4f}")
print(f"{'ResNet-101 (tuned)':<25} {0.5585:>10.4f} {0.5921:>10.4f}")

# Improvement analysis
baseline_f1 = 0.3281
tuned_f1 = m.get('f1', 0)

print()
if tuned_f1 > baseline_f1:
    improvement = ((tuned_f1 - baseline_f1) / baseline_f1) * 100
    print(f"  ✅ EEGNet improved by {improvement:.2f}% with GridSearch!")
elif tuned_f1 == baseline_f1:
    print(f"  ⚠️  EEGNet performance unchanged")
else:
    decline = ((baseline_f1 - tuned_f1) / baseline_f1) * 100
    print(f"  ⚠️  EEGNet declined by {decline:.2f}%")

# Gap to best model
resnet_f1 = 0.5585
gap = resnet_f1 - tuned_f1
gap_pct = (gap / resnet_f1) * 100
print(f"  📉 Gap to ResNet-101: {gap:.4f} ({gap_pct:.1f}%)")

# Convert to DataFrame for detailed analysis
print("\n" + "="*80)
print(" 📊 HYPERPARAMETER IMPACT ANALYSIS ".center(80, "="))
print("="*80)

records = []
for r in all_results:
    record = {
        'config_id': r.get('config_id'),
        **r.get('params', {}), 
        **r.get('mean_metrics', {})
    }
    records.append(record)

df = pd.DataFrame(records)

# Optimizer impact
print("\n🔧 OPTIMIZER PERFORMANCE:")
print("-"*80)
opt_stats = df.groupby('optimizer')['f1'].agg(['mean', 'std', 'max', 'min', 'count'])
opt_stats = opt_stats.sort_values('mean', ascending=False)
print(opt_stats)

# Activation impact
print("\n⚡ ACTIVATION FUNCTION PERFORMANCE:")
print("-"*80)
act_stats = df.groupby('activation')['f1'].agg(['mean', 'std', 'max', 'min', 'count'])
act_stats = act_stats.sort_values('mean', ascending=False)
print(act_stats)

# L2 Regularization impact (L1 is all 0)
print("\n🎛️ L2 REGULARIZATION IMPACT:")
print("-"*80)
l2_stats = df.groupby('l2_lambda')['f1'].agg(['mean', 'std', 'max', 'min', 'count'])
l2_stats = l2_stats.sort_values('mean', ascending=False)
print(l2_stats)

# Combined best combinations
print("\n🎯 BEST COMBINATIONS:")
print("-"*80)
print(df.groupby(['optimizer', 'activation'])['f1'].agg(['mean', 'max']).sort_values('mean', ascending=False))

# Overall statistics
print("\n📈 OVERALL F1 SCORE STATISTICS:")
print("="*80)
print(f"  Best:    {df['f1'].max():.4f} (Config {df.loc[df['f1'].idxmax(), 'config_id']})")
print(f"  Worst:   {df['f1'].min():.4f} (Config {df.loc[df['f1'].idxmin(), 'config_id']})")
print(f"  Mean:    {df['f1'].mean():.4f}")
print(f"  Median:  {df['f1'].median():.4f}")
print(f"  Std Dev: {df['f1'].std():.4f}")
print(f"  Range:   {df['f1'].max() - df['f1'].min():.4f}")

print("\n" + "="*80)
print("✅ Analysis complete!")
print("="*80)

# Show worst performers for comparison
print("\n⚠️  BOTTOM 3 CONFIGURATIONS (for reference):")
print("-"*80)
print(f"{'Rank':<6} {'ID':<5} {'Optimizer':>10} {'Activation':>12} {'L2':>8} {'F1':>10} {'Acc':>8}")
print("-"*80)
for i, result in enumerate(sorted_results[-3:], 1):
    p = result.get('params', {})
    m = result.get('mean_metrics', {})
    config_id = result.get('config_id', 'N/A')
    
    print(f"{len(sorted_results)-3+i:<6} {config_id:<5} {p.get('optimizer', 'N/A'):>10} "
          f"{p.get('activation', 'N/A'):>12} "
          f"{p.get('l2_lambda', 0):>8.0e} "
          f"{m.get('f1', 0):>10.4f} "
          f"{m.get('accuracy', 0):>8.4f}")

print("\n" + "="*80)

🔍 Loading file...
📁 C:\Users\numpppy\Downloads\hms-harmful-brain-activity-classification\eegnet_gridsearch_results\eegnet_gridsearch_final.json

✅ Successfully loaded 18 configurations!

🏆 TOP 10 CONFIGURATIONS:
Rank   ID     Optimizer   Activation       L1       L2         F1      Acc
--------------------------------------------------------------------------------
1      3           adam         relu    0e+00    1e-03     0.3892   0.3731
2      10         adamw    leakyrelu    0e+00    0e+00     0.3887   0.3684
3      5           adam    leakyrelu    0e+00    1e-04     0.3864   0.3657
4      6           adam    leakyrelu    0e+00    1e-03     0.3864   0.3692
5      9          adamw         relu    0e+00    1e-03     0.3837   0.3613
6      12         adamw    leakyrelu    0e+00    1e-03     0.3834   0.3631
7      4           adam    leakyrelu    0e+00    0e+00     0.3813   0.3632
8      1           adam         relu    0e+00    0e+00     0.3811   0.3644
9      7          adamw         